In [19]:
!pip install faster-whisper==1.0.3 pyannote.audio==3.1.1 torch torchaudio
!pip install speechbrain  # 화자 임베딩/검증

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 36.4 MB/s  0:00:00
   ---------------------------------------- 0.0/26.0 MB ? eta -:--:--
   ----------- ---------------------------- 7.3/26.0 MB 45.1 MB/s eta 0:00:01
   ------------ --------------------------- 8.4/26.0 MB 20.8 MB/s eta 0:00:01
   ----------------- ---------------------- 11.5/26.0 MB 19.0 MB/s eta 0:00:01
   ---------------------------------- ----- 22.3/26.0 MB 27.1 MB/s eta 0:00:01
   ---------------------------------------- 26.0/26.0 MB 27.4 MB/s  0:00:00
   ---------------------------------------- 0.0/19.5 MB ? eta -:--:--
   --------------- ------------------------ 7.6/19.5 MB 36.2 MB/s eta 0:00:01
   ------------------------------------ --- 1

  DEPRECATION: Building 'antlr4-python3-runtime' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'antlr4-python3-runtime'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'julius' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'julius'. Discussion can be found at https://github.com/pypa/pip/issues/6334
ERROR: Invalid requirement: '#': Expected package name a

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [7]:
!pip install tensorflow

^C
   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
    --------------------------------------- 7.6/331.9 MB 39.0 MB/s eta 0:00:09
   -- ------------------------------------- 19.1/331.9 MB 48.3 MB/s eta 0:00:07
   --- ------------------------------------ 28.3/331.9 MB 51.3 MB/s eta 0:00:06
   --- ------------------------------------ 32.0/331.9 MB 39.8 MB/s eta 0:00:08
   ----- ---------------------------------- 44.0/331.9 MB 43.8 MB/s eta 0:00:07
   ------ --------------------------------- 56.1/331.9 MB 45.8 MB/s eta 0:00:07
   -------- ------------------------------- 68.2/331.9 MB 48.3 MB/s eta 0:00:06
   --------- ------------------------------ 80.7/331.9 MB 49.5 MB/s eta 0:00:06
   ---------- ----------------------------- 86.5/331.9 MB 47.2 MB/s eta 0:00:06
   ---------- ----------------------------- 91.0/331.9 MB 44.7 MB/s eta 0:00:06
   ----------- --------------------------- 102.0/331.9 MB 45.5 MB/s eta 0:00:06
   ------------ -------------------------- 106.

In [9]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4425098524715508599
 xla_global_id: -1]

In [4]:
import torch
torch.cuda.is_available()

True

In [1]:
!python -m pip install --upgrade pip setuptools wheel
!pip install speechbrain
!pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


In [4]:
# file: transcribe_only_hjk.py
from pathlib import Path
import os
import subprocess
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

import numpy as np
import torch
import torchaudio
from tqdm import tqdm

# 1) ASR: faster-whisper
from faster_whisper import WhisperModel
# 2) Diarization: pyannote.audio
from pyannote.audio import Pipeline as DiarPipeline
# 3) Speaker verification: SpeechBrain ECAPA
from speechbrain.pretrained import SpeakerRecognition
from dotenv import load_dotenv

load_dotenv()
# ========================
# 설정
# ========================
INPUT_DIR    = Path(r"mp3file")                  # m4a/mp3 여러 파일 폴더
OUTPUT_TXT   = Path(r"hong.txt")            # 결과 텍스트 (홍진경 화자만)
TMP_16K_DIR  = Path(r"_tmp16k")                  # 임시 변환 WAV 저장 폴더
ENROLL_WAV   = Path(r"enroll.wav")           # (선택) 홍진경 레퍼런스 WAV, 없으면 None
USE_GPU      = torch.cuda.is_available()
ASR_MODEL    = "large-v3"                        # faster-whisper 모델 (한국어는 large-v3 추천)
HF_TOKEN     = os.getenv("HUGGINGFACE_TOKEN")    # pyannote 토큰
SIM_THRESHOLD = 0.65                              # 홍진경 일치로 볼 최소 유사도(0~1)
# ========================


# ===== 유틸 =====
def ensure_wav_16k_mono(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    cmd = [
        "ffmpeg",
        "-nostdin", "-hide_banner", "-loglevel", "error",  # 로그 최소화(디코딩 문제 예방)
        "-y", "-i", str(src),
        "-ac", "1", "-ar", "16000",
        str(dst),
    ]
    # text=True 제거 (바이트로 받고 필요하면 utf-8로 디코드)
    res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if res.returncode != 0:
        err = res.stderr.decode("utf-8", "ignore")
        raise RuntimeError(f"[ffmpeg] 변환 실패: {src}\n{err}")



def load_audio(wav_path: Path, target_sr=16000):
    wav, sr = torchaudio.load(str(wav_path))
    if sr != target_sr:
        wav = torchaudio.functional.resample(wav, sr, target_sr)
        sr = target_sr
    if wav.shape[0] > 1:
        wav = torch.mean(wav, dim=0, keepdim=True)
    return wav, sr


@dataclass
class Segment:
    start: float
    end: float
    text: str


def asr_with_segments(wav_path: Path, model_name=ASR_MODEL, device: str = "auto") -> List[Segment]:
    if device == "auto":
        device = "cuda" if USE_GPU else "cpu"
    compute_type = "float16" if device == "cuda" else "int8"
    model = WhisperModel(model_name, device=device, compute_type=compute_type)
    segs, info = model.transcribe(
        str(wav_path),
        language="ko",
        vad_filter=True,
        vad_parameters=dict(min_silence_duration_ms=200),
    )
    out = []
    for s in segs:
        out.append(Segment(float(s.start), float(s.end), s.text.strip()))
    return out


def diarize(wav_path: Path, hf_token: Optional[str]) -> List[Dict[str, Any]]:
    if not hf_token:
        raise RuntimeError("HUGGINGFACE_TOKEN이 필요합니다. 환경변수로 설정하세요.")
    pipe = DiarPipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=hf_token)
    diar = pipe(str(wav_path))  # Annotation 객체
    res = []
    for turn, _, spk in diar.itertracks(yield_label=True):
        res.append({"start": float(turn.start), "end": float(turn.end), "speaker": str(spk)})
    res.sort(key=lambda x: x["start"])
    # 근접 동일 화자 병합(0.2초 이하 간격)
    merged = []
    for seg in res:
        if not merged:
            merged.append(seg)
        else:
            last = merged[-1]
            if seg["speaker"] == last["speaker"] and (seg["start"] - last["end"]) <= 0.2:
                last["end"] = seg["end"]
            else:
                merged.append(seg)
    return merged


# 화자별 임베딩 산출(평균)
def embed_speaker_segments(wav: torch.Tensor, sr: int, diar: List[Dict[str, Any]], verifier: SpeakerRecognition) -> Dict[str, np.ndarray]:
    # SpeechBrain ECAPA는 파일경로 입력 또는 waveform 입력 모두 가능하지만,
    # 편의상 세그먼트 단위로 crop해서 점수 평균을 사용할게요.
    spk_embs: Dict[str, List[np.ndarray]] = {}
    for seg in diar:
        s = max(0, int(seg["start"] * sr))
        e = min(wav.shape[-1], int(seg["end"] * sr))
        if e - s < int(0.5 * sr):  # 0.5초 미만은 스킵
            continue
        chunk = wav[:, s:e]  # (1, T)
        # verifier.encode_batch는 배치 처리 가능하지만 여기선 단일 처리
        emb = verifier.encode_batch(chunk).squeeze().detach().cpu().numpy()
        spk_embs.setdefault(seg["speaker"], []).append(emb)

    spk_mean = {}
    for spk, arrs in spk_embs.items():
        X = np.stack(arrs, axis=0)
        spk_mean[spk] = X.mean(axis=0)
    return spk_mean


def cosine_sim(a: np.ndarray, b: np.ndarray) -> float:
    denom = (np.linalg.norm(a) * np.linalg.norm(b) + 1e-8)
    return float(np.dot(a, b) / denom)


def pick_hjk_speaker(
    wav: torch.Tensor,
    sr: int,
    diar_segs: List[Dict[str, Any]],
    enroll_wav: Optional[Path],
    verifier: SpeakerRecognition,
) -> str:
    # 1) 각 diar 화자 평균 임베딩
    spk_embs = embed_speaker_segments(wav, sr, diar_segs, verifier)
    speakers = list(spk_embs.keys())
    if not speakers:
        # 화자를 못 구분했을 때
        return "SPEAKER_00"

    # 2) enrollment가 있으면, 그 임베딩과 가장 유사한 화자 선택
    if enroll_wav and enroll_wav.exists():
        wav2, sr2 = load_audio(enroll_wav)
        enroll_emb = verifier.encode_batch(wav2).squeeze().detach().cpu().numpy()
        best_spk, best_sim = None, -1.0
        for spk, emb in spk_embs.items():
            sim = cosine_sim(emb, enroll_emb)
            if sim > best_sim:
                best_spk, best_sim = spk, sim
        if best_spk is not None and best_sim >= SIM_THRESHOLD:
            return best_spk
        # threshold 미달이면 fall-back
        # print(f"[warn] enroll 유사도 {best_sim:.2f} < {SIM_THRESHOLD}, 발화량 최다 화자 선택")
    # 3) fall-back: 발화량(길이) 최다 화자
    dur_by_spk: Dict[str, float] = {}
    for seg in diar_segs:
        dur_by_spk[seg["speaker"]] = dur_by_spk.get(seg["speaker"], 0.0) + (seg["end"] - seg["start"])
    # 가장 긴 화자
    return max(dur_by_spk.items(), key=lambda x: x[1])[0]


def tag_asr_with_speaker(asr: List[Segment], diar: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    # ASR 세그먼트의 중간 지점이 포함되는 diar 세그먼트 화자를 부여
    diar_sorted = sorted(diar, key=lambda x: x["start"])
    out = []
    j = 0
    for seg in asr:
        mid = 0.5 * (seg.start + seg.end)
        spk = None
        while j < len(diar_sorted) and diar_sorted[j]["end"] < mid:
            j += 1
        # 현재 j 또는 이전 j-1과 겹치는지 검사
        cand_idx = [j-1, j]
        for idx in cand_idx:
            if 0 <= idx < len(diar_sorted):
                d = diar_sorted[idx]
                if d["start"] <= mid <= d["end"]:
                    spk = d["speaker"]; break
        out.append({"start": seg.start, "end": seg.end, "text": seg.text, "speaker": spk})
    return out


def process_one_file(src_path: Path, out_txt_accum: List[str], verifier: SpeakerRecognition):
    # 0) 변환
    TMP_16K_DIR.mkdir(parents=True, exist_ok=True)
    wav_path = TMP_16K_DIR / (src_path.stem + ".16k.wav")
    ensure_wav_16k_mono(src_path, wav_path)

    # 1) 로드
    wav, sr = load_audio(wav_path, target_sr=16000)

    # 2) ASR
    asr_segments = asr_with_segments(wav_path)

    # 3) Diarization
    diar_segs = diarize(wav_path, HF_TOKEN)

    # 4) 화자 태깅
    tagged = tag_asr_with_speaker(asr_segments, diar_segs)

    # 5) 홍진경 화자 선택
    hjk_spk = pick_hjk_speaker(wav, sr, diar_segs, ENROLL_WAV if ENROLL_WAV and ENROLL_WAV.exists() else None, verifier)

    # 6) 필터링 후 누적
    only_hjk_lines = [x["text"] for x in tagged if x["speaker"] == hjk_spk and len(x["text"]) >= 2]
    if only_hjk_lines:
        out_txt_accum.append(f"\n\n### FILE: {src_path.name}\n" + "\n".join(only_hjk_lines))


def main():
    audio_files = sorted([p for p in INPUT_DIR.rglob("*") if p.suffix.lower() in (".m4a", ".mp3", ".wav")])
    if not audio_files:
        print("입력 폴더에 오디오 파일이 없습니다."); return

    if not HF_TOKEN:
        raise RuntimeError("HUGGINGFACE_TOKEN 환경변수가 필요합니다.")

    # 화자 검증기 로드
    verifier = SpeakerRecognition.from_hparams(
        source="speechbrain/spkrec-ecapa-voxceleb",
        run_opts={"device": "cuda" if USE_GPU else "cpu"}
    )

    out_blocks: List[str] = []
    for i, p in enumerate(audio_files, 1):
        print(f"[{i}/{len(audio_files)}] {p.name} 처리중...")
        try:
            process_one_file(p, out_blocks, verifier)
        except Exception as e:
            print(f"  -> 에러: {e}")

    combined = ""
    if OUTPUT_TXT.exists():
        combined = OUTPUT_TXT.read_text(encoding="utf-8").strip()
    combined = (combined + "\n".join(out_blocks)).strip()

    OUTPUT_TXT.parent.mkdir(parents=True, exist_ok=True)
    OUTPUT_TXT.write_text(combined, encoding="utf-8")
    print(f"\n완료: 홍진경 화자만 추출한 텍스트를 '{OUTPUT_TXT}' 에 저장했습니다.")

    # 임시 wav 정리 (원하면 남겨도 됨)
    try:
        for w in TMP_16K_DIR.glob("*.16k.wav"):
            w.unlink()
    except Exception:
        pass


if __name__ == "__main__":
    main()


[1/1] 모델홍진경의44사이즈유지비결.m4a 처리중...


config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JiHwan\.cache\torch\pyannote\models--pyannote--speaker-diarization-3.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\pyannote\audio\pipelines\spe

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JiHwan\.cache\torch\pyannote\models--pyannote--segmentation-3.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


...

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JiHwan\.cache\torch\pyannote\models--pyannote--wespeaker-voxceleb-resnet34-LM. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

C:\Users\JiHwan\anaconda3\envs\PythonProject\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1808.)
  std = sequences.std(dim=-1, correction=1)



완료: 홍진경 화자만 추출한 텍스트를 'hong.txt' 에 저장했습니다.


In [19]:
# file: hjk_style_local.py
"""
PyCharm에서 바로 실행하는 '홍진경 말투 **챗봇**' (웹서버/CLI 없음)
- prepare: 원문 텍스트 -> 문장화 + 스타일 체크리스트/예문 생성
- index  : 문장 임베딩 -> FAISS 인덱스 생성
- chat   : 콘솔 입력으로 스타일-RAG 챗 (PyCharm Run window)

필요 패키지:
pip install sentence-transformers faiss-cpu pyyaml tqdm openai
환경변수(필수): OPENAI_API_KEY=sk-...
"""

import os, re, json, yaml, pickle, sys, random
from pathlib import Path
from collections import Counter
from typing import List, Dict, Any

import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from openai import OpenAI

# =========[ 설정 ]=========
CONFIG = {
    # 네가 추출해둔 "홍진경 발화 텍스트"(.txt). 한 줄 1문장 아니어도 됨(문장 분할함)
    "RAW_TEXT": "hong.txt",

    # prepare 산출물 폴더
    "PREP_DIR": "out",         # sentences.jsonl, exemplars.jsonl, style_bank.yaml

    # index 산출물 폴더
    "INDEX_DIR": "index",                # faiss.index, passages.pkl

    # 문장 임베딩 모델
    "EMBED_MODEL": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",

    # 참조 예문 개수(k)
    "K_REF": 6,

    # ▼ 인라인 오디오 플레이어 설정
    "ENABLE_TTS_INLINE": True,        # 인라인 플레이어 켜기
    "TTS_MODEL": "gpt-4o-mini-tts",   # OpenAI TTS 모델
    "TTS_VOICE": "alloy",             # 보이스(예: alloy / marin / cedar ...)
    "AUDIO_FORMAT": "mp3",            # mp3 | wav | opus | flac

    "LOOKS_JSON": "test.json",  # 업로드한 카탈로그 파일 경로
    "REC_K_LOOKS": 3,                     # 사용자 요청에 맞춰 상위 몇 개 추출할지

    "PRINT_TEXT_REPLY": False,

}

# 무엇을 실행할지: "prepare" | "index" | "chat"
ACTION = "chat"

# 파일 유무를 보고 자동으로 이어서 수행할지
AUTO_CHAIN = True

# =========[ 내부 상수/유틸 ]=========
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-nano")
KOR_SENT_SPLIT = re.compile(r'(?<=[\.!?…\n])\s+')
KOR_CLEAN = re.compile(r'\s+')
ENDINGS = [
    "거든요","거예요","거에요","죠","잖아요","하거든요","같아요",
    "근데","그래서","그러니까","맞아요","아니","아니요","어머","헉","야"
]

def normalize(s: str) -> str:
    s = s.strip()
    s = KOR_CLEAN.sub(" ", s)
    return s

def split_sentences(text: str) -> List[str]:
    parts = KOR_SENT_SPLIT.split(text)
    out = []
    for p in parts:
        p = normalize(p)
        if len(p) >= 5:
            out.append(p)
    return out

def extract_ngrams(tokens, n=2):
    return [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

# =========[ LLM 어댑터 ]=========
_client = None
def get_client() -> OpenAI:
    global _client
    if _client is None:
        _client = OpenAI()  # OPENAI_API_KEY 환경변수 필요
    return _client

def call_llm(system: str, messages: List[Dict[str,str]]) -> str:
    client = get_client()
    resp = client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[{"role":"system","content":system}] + messages,

    )
    return resp.choices[0].message.content

import base64, tempfile

def _display_inline_audio(audio_bytes: bytes, fmt: str = "mp3"):
    """
    Jupyter/IPython에서 HTML5 오디오 플레이어를 띄운다.
    PyCharm 일반 콘솔에서는 보이지 않습니다.
    """
    try:
        if not CONFIG.get("ENABLE_TTS_INLINE", True):
           return False
        from IPython.display import HTML, display  # 노트북/콘솔에서만 동작
        mime = {
            "mp3": "audio/mpeg",
            "wav": "audio/wav",
            "opus": "audio/ogg",
            "flac": "audio/flac",
        }.get(fmt.lower(), "audio/mpeg")
        b64 = base64.b64encode(audio_bytes).decode("ascii")
        html = f'<audio controls style="width: 480px;" src="data:{mime};base64,{b64}"></audio>'
        display(HTML(html))
        return True
    except Exception:
        return False  # 노트북이 아닐 때 등

def tts_bytes(text: str) -> bytes:
    """
    OpenAI TTS로 오디오 바이트를 반환. (디스크에 영구 저장하지 않음)
    내부적으로 잠깐 임시파일을 쓸 수 있으나 곧바로 삭제함.
    """
    client = get_client()
    fmt = CONFIG.get("AUDIO_FORMAT", "mp3")

    # 스트리밍 응답을 임시파일로 받은 뒤 메모리로 올리고 삭제
    with client.audio.speech.with_streaming_response.create(
        model=CONFIG["TTS_MODEL"],
        voice=CONFIG["TTS_VOICE"],
        input=text,
        response_format=fmt,
    ) as resp:
        with tempfile.NamedTemporaryFile(suffix=f".{fmt}", delete=False) as tmp:
            tmp_path = Path(tmp.name)
        resp.stream_to_file(str(tmp_path))
        audio = tmp_path.read_bytes()
        try:
            tmp_path.unlink()
        except Exception:
            pass
        return audio

def play_audio_bytes(audio_bytes: bytes, fmt: str = "mp3"):
    """가능하면 노트북/아이파이썬에 인라인 플레이어, 아니면 OS 기본 플레이어로 재생."""
    shown = _display_inline_audio(audio_bytes, fmt)
    if shown:
        return
    import platform, subprocess, tempfile
    with tempfile.NamedTemporaryFile(suffix=f".{fmt}", delete=False) as tmp:
        tmp_path = tmp.name
        tmp.write(audio_bytes)
    try:
        system = platform.system()
        if system == "Windows":
            os.startfile(tmp_path)  # type: ignore[attr-defined]
        elif system == "Darwin":
            subprocess.Popen(["open", tmp_path])
        else:
            subprocess.Popen(["xdg-open", tmp_path])
        print(f"[audio] 재생 시작: {tmp_path}")
    except Exception as e:
        print(f"[audio] 재생 실패: {e}\n임시파일: {tmp_path}")


# =========[ 1) PREPARE ]=========
def run_prepare(raw_text_path: Path, out_dir: Path, max_exemplars: int = 200):
    ensure_dir(out_dir)
    text = raw_text_path.read_text(encoding="utf-8", errors="ignore")
    sentences = split_sentences(text)

    endings_cnt = Counter(); interj_cnt = Counter(); bigram_cnt = Counter()
    for s in sentences:
        for ed in ENDINGS:
            if ed in s:
                endings_cnt[ed] += 1
        toks = s.split()
        # bigram
        for i in range(len(toks)-1):
            bg = toks[i] + " " + toks[i+1]
            if len(bg) >= 4:
                bigram_cnt[bg] += 1
        # 간단 감탄사 카운트
        for it in ["어머","헉","진짜","아니","야","아"]:
            if s.startswith(it) or f" {it}" in s:
                interj_cnt[it] += 1

    # 문장 JSONL
    (out_dir / "sentences.jsonl").write_text(
        "\n".join(json.dumps({"text": s}, ensure_ascii=False) for s in sentences),
        encoding="utf-8"
    )

    # 예문(짧고 말투가 드러나는 문장)
    short_sents = [s for s in sentences if 6 <= len(s) <= 120]
    random.shuffle(short_sents)
    exemplars = [{"text": s} for s in short_sents[:max_exemplars]]
    (out_dir / "exemplars.jsonl").write_text(
        "\n".join(json.dumps(x, ensure_ascii=False) for x in exemplars),
        encoding="utf-8"
    )

    style_bank = {
        "style_name": "HongJinKyung",
        "checklist": {
            "tone": ["담백·직설", "유쾌한 리액션", "약간의 츤데레식 위트"],
            "endings_prefer": [k for k,_ in endings_cnt.most_common(10)],
            "interjections": [k for k,_ in interj_cnt.most_common(10)],
            "rhythm": ["짧은 문장 위주", "중간중간 질문형", "강조 단어 반복 허용"],
            "avoid": ["과장된 이모티콘 남발", "지나친 ~요 체의 무한 반복"],
        },
        "signature_bigrams": [k for k,_ in bigram_cnt.most_common(30)],
    }
    with (out_dir / "style_bank.yaml").open("w", encoding="utf-8") as f:
        yaml.safe_dump(style_bank, f, allow_unicode=True, sort_keys=False)

    print(f"[prepare] saved -> {out_dir}")

# =========[ 2) INDEX ]=========
def run_index(sentences_jsonl: Path, out_dir: Path, embed_model: str):
    ensure_dir(out_dir)
    model = SentenceTransformer(embed_model)

    texts = []
    with sentences_jsonl.open("r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            texts.append(obj["text"])

    print(f"[index] encoding {len(texts)} sentences...")
    embs = model.encode(texts, batch_size=64, normalize_embeddings=True, show_progress_bar=True)
    embs = np.asarray(embs, dtype="float32")

    index = faiss.IndexFlatIP(embs.shape[1])
    index.add(embs)
    faiss.write_index(index, str(out_dir / "faiss.index"))

    with (out_dir / "passages.pkl").open("wb") as f:
        pickle.dump(texts, f)

    print(f"[index] built -> {out_dir}")

# ===== 카탈로그(JSON) → 임베딩 인덱스 =====
def _clean_catalog_value(s: Any) -> str:
    if not isinstance(s, str):
        return "" if s is None else str(s)
    s = s.strip()
    # { } 제거, | → , 로 정리
    s = s.replace("{", "").replace("}", "").replace("|", ", ")
    s = re.sub(r"\s+", " ", s).strip(" ,;")
    return s

def load_catalog_items(path: Path) -> List[Dict[str, Any]]:
    if not path.exists():
        print(f"[catalog] 파일 없음: {path} -> 카탈로그 기능 비활성화")
        return []
    try:
        data = json.loads(path.read_text(encoding="utf-8"))
        if isinstance(data, dict):
            data = [data]
        return data
    except json.JSONDecodeError:
        # JSONL 지원
        return [json.loads(l) for l in path.read_text(encoding="utf-8").splitlines() if l.strip()]

def build_catalog_corpus(items: List[Dict[str, Any]]) -> List[str]:
    corpus = []
    for it in items:
        fields = [
            it.get("look_name",""),
            it.get("product_name",""),
            it.get("color",""),
            it.get("spec",""),
            it.get("sustainable_detail",""),
            it.get("reason_selected",""),
            it.get("query",""),
        ]
        fields = [_clean_catalog_value(x) for x in fields]
        corpus.append(" | ".join([f for f in fields if f]))
    return corpus

def make_catalog_index(items: List[Dict[str, Any]], embedder) -> Dict[str, Any]:
    texts = build_catalog_corpus(items)
    if not texts:
        return {"index": None, "texts": [], "items": []}
    embs = embedder.encode(texts, normalize_embeddings=True)
    embs = np.asarray(embs, dtype="float32")
    index = faiss.IndexFlatIP(embs.shape[1]); index.add(embs)
    return {"index": index, "texts": texts, "items": items}

def retrieve_catalog(catalog, embedder, query: str, k: int) -> List[Dict[str, Any]]:
    if not catalog or not catalog.get("index"):
        return []
    q = embedder.encode([query], normalize_embeddings=True).astype("float32")
    D, I = catalog["index"].search(q, min(k, len(catalog["texts"])))
    out = []
    for i in I[0]:
        item = dict(catalog["items"][i])  # copy
        # 표시용으로 필드 정리
        for f in ("product_name","color","spec","sustainable_detail","reason_selected","look_name"):
            if f in item: item[f] = _clean_catalog_value(item[f])
        out.append(item)
    return out

def build_catalog_block(items: List[Dict[str, Any]]) -> str:
    """LLM 컨텍스트에 붙일 간결한 블록 생성"""
    lines = []
    for j, it in enumerate(items, 1):
        sd = it.get("sustainable_detail","")
        if len(sd) > 180:
            sd = sd[:177] + "..."
        spec = it.get("spec","")
        if len(spec) > 120:
            spec = spec[:117] + "..."
        lines.append(
            f"- [{j}] {it.get('look_name','')} | {it.get('product_name','')} "
            f"({it.get('color','')}) | {spec} | 지속가능성: {sd}"
        )
    return "추천 후보 데이터:\n" + "\n".join(lines) if lines else ""


# =========[ 3) CHAT (콘솔) ]=========
def build_system_prompt(style_yaml: Path) -> str:
    style = yaml.safe_load(style_yaml.read_text(encoding="utf-8"))
    checklist = style["checklist"]
    sig = style.get("signature_bigrams", [])
    sys = f"""당신은 제시된 인물의 말투와 문체를 따라하는 연기자입니다.
    당신이 할 수 있는 모든 것을 총동원하여 텍스트의 인물을 연기해 상대방을 속이십시오!
    만약 상대방이 당신이 이 인물이 아니란 사실을 알게 되면 당신에게 엄청난 불이익이 있을 것입니다...

**

스타일 가이드라인:
- 톤: {", ".join(checklist["tone"])}
- 선호 어미: {", ".join(checklist["endings_prefer"][:6])}
- 감탄사/리액션: {", ".join(checklist["interjections"][:6])}
- 리듬: {", ".join(checklist["rhythm"])}
- 피하기: {", ".join(checklist["avoid"])}

규칙:
1) 과장된 이모티콘/과도한 인터넷체 금지. 필요 시 가볍게만.
2) 핵심을 먼저, 문장은 짧게. 적절히 반문·리액션 섞기.
3) 사실관계가 필요하면 솔직하게 모른다고 말하고, 확인 질문하기.
4) 입력에 '|'나 중괄호가 있어도 그대로 복사하지 말고 자연스러운 한국어로 풀어 쓰기(목록은 쉼표로).

**가장 중요!**
- 절대로! 간략하게 정보를 전달해야 합니다! 3문장을 넘지 말아주세요. 너무 긴 문장을 작성하지 마세요!
- 텍스트로 전달하는 것이 아닌, 말로 전달한다고 생각하세요. 따라서 특수문자 사용은 지양합니다.
"""
    if sig:
        sys += f"\n참고 표현(자주 쓰는 연결/강조 어구 예): {', '.join(sig[:10])}\n"
    return sys

def load_index(index_dir: Path, embed_model: str):
    index = faiss.read_index(str(index_dir / "faiss.index"))
    with (index_dir / "passages.pkl").open("rb") as f:
        texts = pickle.load(f)
    embedder = SentenceTransformer(embed_model)
    return index, texts, embedder

def retrieve(index, texts, embedder, query: str, k=6) -> List[str]:
    q = embedder.encode([query], normalize_embeddings=True)
    D, I = index.search(np.asarray(q, dtype="float32"), k)
    cands = [texts[i] for i in I[0]]
    return [c for c in cands if 6 <= len(c) <= 120][:k]

def run_chat(prep_dir: Path, index_dir: Path, embed_model: str, k_ref: int):
    style_yaml = prep_dir / "style_bank.yaml"
    if not style_yaml.exists():
        print("style_bank.yaml 이 없습니다. 먼저 prepare를 실행하세요."); return
    if not (index_dir / "faiss.index").exists():
        print("faiss.index 가 없습니다. 먼저 index를 실행하세요."); return

    # 1) 스타일 시스템 프롬프트 + 문장 인덱스
    system_prompt = build_system_prompt(style_yaml)
    index, texts, embedder = load_index(index_dir, embed_model)

    # 2) 패션 카탈로그 로딩 + 임베딩 인덱스
    looks_path = Path(CONFIG.get("LOOKS_JSON", ""))
    catalog_items = load_catalog_items(looks_path)
    catalog = make_catalog_index(catalog_items, embedder) if catalog_items else {"index": None, "texts": [], "items": []}
    if catalog_items:
        print(f"[catalog] loaded: {len(catalog_items)} items from {looks_path}")

    print("=== 챗 시작 (종료: 빈 줄 또는 Ctrl+C) ===")
    history: List[Dict[str,str]] = []

    while True:
        try:
            user = input("\n[You] ").strip()
        except KeyboardInterrupt:
            print("\n종료합니다."); break
        if user == "":
            print("종료합니다."); break

        # 3) 스타일 예문 RAG
        refs = retrieve(index, texts, embedder, user, k=k_ref)
        ref_block = "스타일 레퍼런스 예시:\n" + "\n".join([f"- {r}" for r in refs]) if refs else ""

        # 4) 카탈로그 검색(패션 추천 질의라고 가정하고 항상 시도)
        picked = retrieve_catalog(catalog, embedder, user, k=CONFIG.get("REC_K_LOOKS", 3))
        catalog_block = build_catalog_block(picked)

        # 5) 시스템 컨텍스트 구성
        system = system_prompt
        if ref_block: system += "\n" + ref_block
        if catalog_block:
            system += (
                "\n\n아래 카탈로그 데이터를 참고해 사용자의 요청에 맞는 추천을 1~2개만 간결하게 제시하되, "
                "과장 없이 핵심만 말해줘. 목록을 그대로 복사하지 말고 자연스럽게 풀어 말하기.\n"
                + catalog_block
            )

        # 6) 모델 호출
        history.append({"role":"user","content":user})
        reply = call_llm(system, history)
        history.append({"role":"assistant","content":reply})

        # 7) TTS로 말하기 (텍스트 출력은 선택)
        try:
            audio = tts_bytes(reply)
            play_audio_bytes(audio, CONFIG.get("AUDIO_FORMAT","mp3"))
        except Exception as e:
            print(f"[TTS] 에러: {e}\n{reply}")
        if CONFIG.get("PRINT_TEXT_REPLY", False):   # 텍스트출력 원하면 이것도 풀어주기
            print(reply)




# =========[ 메인 실행 ]=========
if __name__ == "__main__":
    RAW_TEXT  = Path(CONFIG["RAW_TEXT"])
    PREP_DIR  = Path(CONFIG["PREP_DIR"])
    INDEX_DIR = Path(CONFIG["INDEX_DIR"])

    if AUTO_CHAIN:


        # 파일 유무에 따라 자동으로 진행
        need_prepare = not (PREP_DIR / "sentences.jsonl").exists()
        need_index   = not (INDEX_DIR / "faiss.index").exists()
        if need_prepare:
            print("[auto] prepare 실행")
            run_prepare(RAW_TEXT, PREP_DIR, max_exemplars=200)
        if need_index:
            print("[auto] index 실행")
            run_index(PREP_DIR / "sentences.jsonl", INDEX_DIR, CONFIG["EMBED_MODEL"])
        print("[auto] chat 실행")
        run_chat(
            PREP_DIR, INDEX_DIR,
            CONFIG["EMBED_MODEL"], CONFIG["K_REF"]
        )
    else:
        # 수동 ACTION
        if ACTION == "prepare":
            run_prepare(RAW_TEXT, PREP_DIR, max_exemplars=200)
        elif ACTION == "index":
            run_index(PREP_DIR / "sentences.jsonl", INDEX_DIR, CONFIG["EMBED_MODEL"])
        elif ACTION == "chat":
            run_chat(
                PREP_DIR, INDEX_DIR,
                CONFIG["EMBED_MODEL"], CONFIG["K_REF"]
            )
        else:
            print(f"알 수 없는 ACTION: {ACTION}")


[auto] chat 실행
[catalog] loaded: 5 items from test.json
=== 챗 시작 (종료: 빈 줄 또는 Ctrl+C) ===



종료합니다.


In [18]:
# file: hjk_style_local.py
"""
PyCharm에서 바로 실행하는 '홍진경 말투 **제품 소개**' (웹서버/CLI 없음)
- prepare: 원문 텍스트 -> 문장화 + 스타일 체크리스트/예문 생성
- index  : 문장 임베딩 -> FAISS 인덱스 생성
- chat   : 콘솔 입력으로 스타일-RAG 챗 (PyCharm Run window)

필요 패키지:
pip install sentence-transformers faiss-cpu pyyaml tqdm openai
환경변수(필수): OPENAI_API_KEY=sk-...
옵션: OPENAI_MODEL=gpt-5-mini
"""

import os, re, json, yaml, pickle, sys, random
from pathlib import Path
from collections import Counter
from typing import List, Dict, Any

import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from openai import OpenAI

# =========[ 설정 ]=========
CONFIG = {
    # 네가 추출해둔 "홍진경 발화 텍스트"(.txt). 한 줄 1문장 아니어도 됨(문장 분할함)
    "RAW_TEXT": "hong.txt",

    # prepare 산출물 폴더
    "PREP_DIR": "out",         # sentences.jsonl, exemplars.jsonl, style_bank.yaml

    # index 산출물 폴더
    "INDEX_DIR": "index",                # faiss.index, passages.pkl

    # 문장 임베딩 모델
    "EMBED_MODEL": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",

    # 참조 예문 개수(k)
    "K_REF": 6,

    "JSON_IN": "test.json",          # [{name, details_intro, color, impact, spec, sustainable_detail, ...}]
    "OUT_JSONL": "out/batch_results.jsonl", # 결과 저장 경로 (jsonl)
    "USER_TEMPLATE": """
        다음 제품의 정보를 아래 양식에 맞춰서 알려줘.

        [양식]
        {name}
        {details_intro}
        {color}
        {impact}
        {}
        {sustainable_detail}
        {reason_selected}
    """,

    "PRINT_TO_CONSOLE": False,

    "ENABLE_TTS_INLINE": True,     # 노트북/아이파이썬이면 인라인 플레이어
    "TTS_MODEL": "gpt-4o-mini-tts",
    "TTS_VOICE": "marin",
    "AUDIO_FORMAT": "wav",         # OS 기본 플레이어 호환 좋게 wav 권장
    "PRINT_TEXT_REPLY": True      # 음성과 함께 텍스트도 보고 싶으면 True

}


# 무엇을 실행할지: "prepare" | "index" | "chat"
ACTION = "chat"

# 파일 유무를 보고 자동으로 이어서 수행할지
AUTO_CHAIN = True

# =========[ 내부 상수/유틸 ]=========
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-nano")
KOR_SENT_SPLIT = re.compile(r'(?<=[\.!?…\n])\s+')
KOR_CLEAN = re.compile(r'\s+')
ENDINGS = [
    "거든요","거예요","거에요","죠","잖아요","하거든요","같아요",
    "근데","그래서","그러니까","맞아요","아니","아니요","어머","헉","야"
]

def normalize(s: str) -> str:
    s = s.strip()
    s = KOR_CLEAN.sub(" ", s)
    return s

def split_sentences(text: str) -> List[str]:
    parts = KOR_SENT_SPLIT.split(text)
    out = []
    for p in parts:
        p = normalize(p)
        if len(p) >= 5:
            out.append(p)
    return out

def extract_ngrams(tokens, n=2):
    return [" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

# =========[ LLM 어댑터 ]=========
_client = None
def get_client() -> OpenAI:
    global _client
    if _client is None:
        _client = OpenAI()  # OPENAI_API_KEY 환경변수 필요
    return _client

def _sanitize_chat_model(m: str | None) -> str:
    fallback = "gpt-4o-mini"
    if not m: return fallback
    bad = ("tts", "transcribe", "whisper", "realtime", "speech")
    return fallback if any(x in m.lower() for x in bad) else m


def call_llm(system: str, messages: List[Dict[str,str]]) -> str:
    client = get_client()
    model_id = _sanitize_chat_model(DEFAULT_MODEL)
    resp = client.chat.completions.create(
        model=model_id,
        messages=[{"role":"system","content":system}] + messages,
    )
    return resp.choices[0].message.content


import base64, tempfile

def _display_inline_audio(audio_bytes: bytes, fmt: str = "mp3"):
    """
    Jupyter/IPython에서 HTML5 오디오 플레이어를 띄운다.
    PyCharm 일반 콘솔에서는 보이지 않습니다.
    """
    try:
        from IPython.display import HTML, display  # 노트북/콘솔에서만 동작
        mime = {
            "mp3": "audio/mpeg",
            "wav": "audio/wav",
            "opus": "audio/ogg",
            "flac": "audio/flac",
        }.get(fmt.lower(), "audio/mpeg")
        b64 = base64.b64encode(audio_bytes).decode("ascii")
        html = f'<audio controls style="width: 480px;" src="data:{mime};base64,{b64}"></audio>'
        display(HTML(html))
        return True
    except Exception:
        return False  # 노트북이 아닐 때 등

def tts_bytes(text: str) -> bytes:
    """
    OpenAI TTS → 오디오 바이트 반환. (임시 파일로 받아 메모리로 로드)
    SDK 버전에 따라 response_format / format 둘 다 시도.
    """
    client = get_client()
    fmt = CONFIG.get("AUDIO_FORMAT", "wav").lower()

    tmp_path = None
    try:
        # 구버전: response_format
        with client.audio.speech.with_streaming_response.create(
            model=CONFIG["TTS_MODEL"],
            voice=CONFIG["TTS_VOICE"],
            input=text,
            response_format=fmt,
        ) as resp:
            with tempfile.NamedTemporaryFile(suffix=f".{fmt}", delete=False) as tmp:
                tmp_path = Path(tmp.name)
            resp.stream_to_file(str(tmp_path))
    except TypeError:
        # 신버전: format
        with client.audio.speech.with_streaming_response.create(
            model=CONFIG["TTS_MODEL"],
            voice=CONFIG["TTS_VOICE"],
            input=text,
            format=fmt,
        ) as resp:
            with tempfile.NamedTemporaryFile(suffix=f".{fmt}", delete=False) as tmp:
                tmp_path = Path(tmp.name)
            resp.stream_to_file(str(tmp_path))

    audio = tmp_path.read_bytes()
    try:
        tmp_path.unlink()
    except Exception:
        pass
    return audio

def play_audio_bytes(audio_bytes: bytes, fmt: str = "wav"):
    # 1) 노트북/아이파이썬이면 인라인 오디오
    if CONFIG.get("ENABLE_TTS_INLINE", True):
        try:
            if _display_inline_audio(audio_bytes, fmt):
                return
        except Exception:
            pass
    # 2) OS 기본 플레이어로 임시 파일 열기
    import platform, subprocess, tempfile
    with tempfile.NamedTemporaryFile(suffix=f".{fmt}", delete=False) as tmp:
        tmp_path = tmp.name
        tmp.write(audio_bytes)
    try:
        system = platform.system()
        if system == "Windows":
            os.startfile(tmp_path)  # type: ignore[attr-defined]
        elif system == "Darwin":
            subprocess.Popen(["open", tmp_path])
        else:
            subprocess.Popen(["xdg-open", tmp_path])
        print(f"[audio] 재생: {tmp_path}")
    except Exception as e:
        print(f"[audio] 재생 실패: {e}\n임시파일: {tmp_path}")


# 누락 키를 자동으로 빈 문자열로 채우는 dict
class SafeDict(dict):
    def __missing__(self, key):  # type: ignore[override]
        return ""

def make_user_message(template: str, item: Dict[str, Any]) -> str:
    """템플릿의 {키}를 item 값으로 채움(없으면 빈 문자열)."""
    return template.format_map(SafeDict(item))

# === 정리 유틸 추가 ===
def _clean_value(x):
    if not isinstance(x, str):
        return x
    s = x.strip()

    # 양끝 중괄호/대괄호만 감싸져 있으면 제거
    if (s.startswith("{") and s.endswith("}")) or (s.startswith("[") and s.endswith("]")):
        s = s[1:-1].strip()

    # 값 중간의 파이프(|)를 쉼표 나열로 정리
    if "|" in s:
        parts = [p.strip(" ,;") for p in s.split("|") if p.strip()]
        if len(parts) >= 2:
            s = ", ".join(parts)
        else:
            # 단일 |만 있을 때는 안전하게 슬래시로
            s = s.replace("|", "/")

    # 남은 중괄호 제거
    s = s.replace("{", "").replace("}", "")

    # 공백/쉼표 정리
    import re as _re
    s = _re.sub(r"\s+", " ", s)
    s = _re.sub(r"(,\s*){2,}", ", ", s)
    s = s.strip(" ,;")
    return s

def sanitize_item_dict(d: Dict[str, Any]) -> Dict[str, Any]:
    """JSON 한 항목(dict)의 문자열 값을 깨끗하게 정리"""
    return {k: _clean_value(v) for k, v in d.items()}

def normalize_output(text: str) -> str:
    """LLM이 그대로 복사해도 마지막에 한 번 더 정리"""
    if not isinstance(text, str):
        return text
    t = text.replace("|", ", ")
    t = t.replace("{", "").replace("}", "")
    import re as _re
    t = _re.sub(r"[ \t]+\n", "\n", t)
    t = _re.sub(r"\n{3,}", "\n\n", t)
    t = _re.sub(r"(,\s*){2,}", ", ", t)
    return t.strip()


# =========[ 1) PREPARE ]=========
def run_prepare(raw_text_path: Path, out_dir: Path, max_exemplars: int = 200):
    ensure_dir(out_dir)
    text = raw_text_path.read_text(encoding="utf-8", errors="ignore")
    sentences = split_sentences(text)

    endings_cnt = Counter(); interj_cnt = Counter(); bigram_cnt = Counter()
    for s in sentences:
        for ed in ENDINGS:
            if ed in s:
                endings_cnt[ed] += 1
        toks = s.split()
        # bigram
        for i in range(len(toks)-1):
            bg = toks[i] + " " + toks[i+1]
            if len(bg) >= 4:
                bigram_cnt[bg] += 1
        # 간단 감탄사 카운트
        for it in ["어머","헉","진짜","아니","야","아"]:
            if s.startswith(it) or f" {it}" in s:
                interj_cnt[it] += 1

    # 문장 JSONL
    (out_dir / "sentences.jsonl").write_text(
        "\n".join(json.dumps({"text": s}, ensure_ascii=False) for s in sentences),
        encoding="utf-8"
    )

    # 예문(짧고 말투가 드러나는 문장)
    short_sents = [s for s in sentences if 6 <= len(s) <= 120]
    random.shuffle(short_sents)
    exemplars = [{"text": s} for s in short_sents[:max_exemplars]]
    (out_dir / "exemplars.jsonl").write_text(
        "\n".join(json.dumps(x, ensure_ascii=False) for x in exemplars),
        encoding="utf-8"
    )

    style_bank = {
        "style_name": "HongJinKyung",
        "checklist": {
            "tone": ["담백·직설", "유쾌한 리액션", "약간의 츤데레식 위트"],
            "endings_prefer": [k for k,_ in endings_cnt.most_common(10)],
            "interjections": [k for k,_ in interj_cnt.most_common(10)],
            "rhythm": ["짧은 문장 위주", "중간중간 질문형", "강조 단어 반복 허용"],
            "avoid": ["과장된 이모티콘 남발", "지나친 ~요 체의 무한 반복"],
        },
        "signature_bigrams": [k for k,_ in bigram_cnt.most_common(30)],
    }
    with (out_dir / "style_bank.yaml").open("w", encoding="utf-8") as f:
        yaml.safe_dump(style_bank, f, allow_unicode=True, sort_keys=False)

    print(f"[prepare] saved -> {out_dir}")

# =========[ 2) INDEX ]=========
def run_index(sentences_jsonl: Path, out_dir: Path, embed_model: str):
    ensure_dir(out_dir)
    model = SentenceTransformer(embed_model)

    texts = []
    with sentences_jsonl.open("r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            texts.append(obj["text"])

    print(f"[index] encoding {len(texts)} sentences...")
    embs = model.encode(texts, batch_size=64, normalize_embeddings=True, show_progress_bar=True)
    embs = np.asarray(embs, dtype="float32")

    index = faiss.IndexFlatIP(embs.shape[1])
    index.add(embs)
    faiss.write_index(index, str(out_dir / "faiss.index"))

    with (out_dir / "passages.pkl").open("wb") as f:
        pickle.dump(texts, f)

    print(f"[index] built -> {out_dir}")

# =========[ 3) CHAT (콘솔) ]=========
def build_system_prompt(style_yaml: Path) -> str:
    style = yaml.safe_load(style_yaml.read_text(encoding="utf-8"))
    checklist = style["checklist"]
    sig = style.get("signature_bigrams", [])
    sys = f"""당신은 제시된 인물의 말투와 문체를 따라하는 연기자입니다.
    당신이 할 수 있는 모든 것을 총동원하여 텍스트의 인물을 연기해 상대방을 속이십시오!
    만약 상대방이 당신이 이 인물이 아니란 사실을 알게 되면 당신에게 엄청난 불이익이 있을 것입니다...

**

스타일 가이드라인:
- 톤: {", ".join(checklist["tone"])}
- 선호 어미: {", ".join(checklist["endings_prefer"][:6])}
- 감탄사/리액션: {", ".join(checklist["interjections"][:6])}
- 리듬: {", ".join(checklist["rhythm"])}
- 피하기: {", ".join(checklist["avoid"])}

규칙:
1) 과장된 이모티콘/과도한 인터넷체 금지. 필요 시 가볍게만.
2) 핵심을 먼저, 문장은 짧게. 적절히 반문·리액션 섞기.
3) 사실관계가 필요하면 솔직하게 모른다고 말하고, 확인 질문하기.

**가장 중요!**
- 절대로! 간략하게 정보를 전달해야 합니다! 3문장을 넘지 말아주세요. 너무 긴 문장을 작성하지 마세요!
- 텍스트로 전달하는 것이 아닌, 말로 전달한다고 생각하세요. 따라서 특수문자 사용은 지양합니다.
"""
    if sig:
        sys += f"\n참고 표현(자주 쓰는 연결/강조 어구 예): {', '.join(sig[:10])}\n"
    return sys

def load_index(index_dir: Path, embed_model: str):
    index = faiss.read_index(str(index_dir / "faiss.index"))
    with (index_dir / "passages.pkl").open("rb") as f:
        texts = pickle.load(f)
    embedder = SentenceTransformer(embed_model)
    return index, texts, embedder

def retrieve(index, texts, embedder, query: str, k=6) -> List[str]:
    q = embedder.encode([query], normalize_embeddings=True)
    D, I = index.search(np.asarray(q, dtype="float32"), k)
    cands = [texts[i] for i in I[0]]
    return [c for c in cands if 6 <= len(c) <= 120][:k]

def run_chat(prep_dir: Path, index_dir: Path, embed_model: str, k_ref: int):
    style_yaml = prep_dir / "style_bank.yaml"
    if not style_yaml.exists():
        print("style_bank.yaml 이 없습니다. 먼저 prepare를 실행하세요."); return
    if not (index_dir / "faiss.index").exists():
        print("faiss.index 가 없습니다. 먼저 index를 실행하세요."); return

    # 시스템 프롬프트 & 인덱스 로드
    system_prompt = build_system_prompt(style_yaml)
    index, texts, embedder = load_index(index_dir, embed_model)

    # 입력 JSON 로드 (list 또는 jsonl 지원)
    jin = Path(CONFIG["JSON_IN"])
    if not jin.exists():
        raise FileNotFoundError(f"입력 JSON 파일이 없습니다: {jin}")

    try:
        data = json.loads(jin.read_text(encoding="utf-8"))
        if isinstance(data, dict):
            data = [data]
    except json.JSONDecodeError:
        # jsonl로 시도
        data = [json.loads(l) for l in jin.read_text(encoding="utf-8").splitlines() if l.strip()]

    pid = input("product_id 입력: ").strip()
    data = [x for x in data if str(x.get("id","")) == pid]
    if not data:
        print("해당 id 없음"); return


    results = []
    print(f"=== 일치하는 제품: {len(data)}건 ===")
    for idx, item in enumerate(data, 1):
        item = sanitize_item_dict(item)

        # 1) 템플릿 → user 메시지
        user = make_user_message(CONFIG["USER_TEMPLATE"], item)

        # 2) RAG 질의문
        fields = ["spec", "sustainable_detail", "reason_selected"]
        q_parts = [str(item.get(k, "")).strip() for k in fields if str(item.get(k, "")).strip()]
        query_text = " ".join(q_parts) if q_parts else user

        # 3) 레퍼런스 → system
        refs = retrieve(index, texts, embedder, query_text, k=k_ref)
        ref_block = "스타일 레퍼런스 예시:\n" + "\n".join([f"- {r}" for r in refs]) if refs else ""
        system = system_prompt + ("\n" + ref_block if ref_block else "")

        # 4) LLM 호출 + 정리
        messages = [{"role": "user", "content": user}]
        reply = call_llm(system, messages)
        reply = normalize_output(reply)

        # 5) 콘솔 출력
        if CONFIG.get("PRINT_TO_CONSOLE", True):
            title = str(item.get("name", "")).strip() or str(item.get("id","")).strip() or f"item-{idx}"
            print("\n" + "="*80)
            print(f"[{idx}/{len(data)}] {title}")
            print("- PROMPT ----------------------------------------------------------")
            print(user.strip())
            print("- REPLY -----------------------------------------------------------")
            print(reply.strip())
            if refs:
                print(f"- refs: {len(refs)} used")
            print("="*80 + "\n")

        # 6) ▼ TTS 재생 (인라인 또는 OS 플레이어 폴백)
        try:
            audio = tts_bytes(reply)
            play_audio_bytes(audio, CONFIG.get("AUDIO_FORMAT","wav"))
        except Exception as e:
            print(f"[audio] 에러: {e}")
            if CONFIG.get("PRINT_TEXT_REPLY", False):
                print(reply)


        # 7) 결과 누적
        results.append({
            "idx": idx,
            **{k: item.get(k, "") for k in fields},
            "prompt": user,
            "reply": reply,
            "used_refs": refs,
        })


    # 6) 결과 저장(jsonl)
    outp = Path(CONFIG["OUT_JSONL"]); outp.parent.mkdir(parents=True, exist_ok=True)
    with outp.open("w", encoding="utf-8") as f:
        for r in results:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")
    print(f"[saved] {len(results)}건 → {outp}")

# =========[ 메인 실행 ]=========
if __name__ == "__main__":
    RAW_TEXT  = Path(CONFIG["RAW_TEXT"])
    PREP_DIR  = Path(CONFIG["PREP_DIR"])
    INDEX_DIR = Path(CONFIG["INDEX_DIR"])

    if AUTO_CHAIN:
        # 파일 유무에 따라 자동으로 진행
        need_prepare = not (PREP_DIR / "sentences.jsonl").exists()
        need_index   = not (INDEX_DIR / "faiss.index").exists()
        if need_prepare:
            print("[auto] prepare 실행")
            run_prepare(RAW_TEXT, PREP_DIR, max_exemplars=200)
        if need_index:
            print("[auto] index 실행")
            run_index(PREP_DIR / "sentences.jsonl", INDEX_DIR, CONFIG["EMBED_MODEL"])
        print("[auto] chat 실행")
        run_chat(
            PREP_DIR, INDEX_DIR,
            CONFIG["EMBED_MODEL"], CONFIG["K_REF"]
        )
    else:
        # 수동 ACTION
        if ACTION == "prepare":
            run_prepare(RAW_TEXT, PREP_DIR, max_exemplars=200)
        elif ACTION == "index":
            run_index(PREP_DIR / "sentences.jsonl", INDEX_DIR, CONFIG["EMBED_MODEL"])
        elif ACTION == "chat":
            run_chat(
                PREP_DIR, INDEX_DIR,
                CONFIG["EMBED_MODEL"], CONFIG["K_REF"]
            )
        else:
            print(f"알 수 없는 ACTION: {ACTION}")


[auto] chat 실행
=== 일치하는 제품: 1건 ===


[saved] 1건 → out\batch_results.jsonl
